<a href="https://colab.research.google.com/github/karchx/spark/blob/main/spark-t.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset create CSV with pyspark

In [ ]:
!pip install pyspark pandas
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
import requests
import json

data = requests.get("https://www.defense.gov/data.json")
data = data.json()
with open("/tmp/ds_w.json", "w") as f:
  f.write(json.dumps(data["dataset"]))

In [3]:
# !cat /tmp/ds.json
!ls -l --block-size=M /tmp/ds_w.json

-rw-r--r-- 1 root root 1M Mar 13 04:16 /tmp/ds_w.json


In [4]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession

#conf = SparkConf().setMaster("local").setAppName("create_csv")

# Inicializo el Spark Context
spark = SparkSession.builder.master("local").appName("create_csv").getOrCreate()

#spark = SparkSession.builder.master("local").appName("create_csv").getOrCreate()
#spark = SQLContext(sc)

In [ ]:

df = spark.createDataFrame(data["dataset"])
#df.toPandas().to_csv(index=False)
#data = (str(data["dataset"]))
#sc = spark.sparkContext

#df.select("id").show()
df.
df.select('publisher.name').show()

In [ ]:
from pyspark.sql.functions import size

df = spark.createDataFrame(data["dataset"])

df.select('publisher.name', 'keyword', size("keyword")).show()

+--------------------+--------------------+-------------+
|                name|             keyword|size(keyword)|
+--------------------+--------------------+-------------+
|Department of Def...|[Enlistment, Acce...|            2|
|Department of Def...|[Enlistment, Acce...|            2|
|Department of Def...|[Enlistment, Acce...|            2|
|Department of Def...|[Enlistment, Acce...|            2|
|Department of Def...|[Enlistment, Acce...|            2|
|Department of Def...|[Enlistment, Acce...|            2|
|Department of Def...|[Enlistment, Acce...|            2|
|Department of Def...|[Enlistment, Acce...|            2|
|Department of Def...|[Enlistment, Acce...|            2|
|Department of Def...|[Enlistment, Acce...|            2|
|Department of Def...|[Enlistment, Acce...|            2|
|Department of Def...|[Active, Air Forc...|           20|
|Office of the Sec...|[afsc, air-force,...|            3|
|Office of the Sec...|[usaf, air-force,...|            5|
|Department of

In [ ]:
!cat /tmp/ds.csv

Goal

Create a silver layer of data that performs some transformations on the bronze data.

This data should provide two new tables:

providers
the unique name of each provider
keywords
how often each keyword occurred for each provider
Requirements
The following data should exist in the silver layer
providers
id: integer
name: string

keywords
id: integer
provider_id: string
keyword: string
count: integer

Any assumptions about the data, process, etc. are captured in the comments on this story
Any additional testing or capabilities are captured in the comments on this story
Blockers
This story requires that bronze data already exists